In [16]:
import numpy as np
import tensorflow as tf
import pandas as pd
import json
import pickle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [17]:
model = tf.keras.models.load_model("tourism_recommender.h5")

In [18]:
with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [19]:
with open("label_encoders.pkl", "rb") as f:
    label_encoders = pickle.load(f)

In [20]:
with open("destination_mapping.json", "r") as f:
    destination_mapping = json.load(f)
reverse_mapping = {v: k for k, v in destination_mapping.items()}

In [21]:
def preprocess_input(user_input):
    categorical_cols = ["Climate Type", "Best Visiting Season", "Budget Level", "Crime Index"]
    numeric_cols = ["User Ratings (out of 5)", "Social Media Mentions", "Number of Reviews"]
    
    processed_input = {}

    # Encode categorical inputs safely
    for col in categorical_cols:
        if user_input[col] in label_encoders[col].classes_:
            processed_input[col] = label_encoders[col].transform([user_input[col]])[0]
        else:
            print(f"Warning: '{user_input[col]}' is not in training data for {col}. Assigning most frequent category.")
            processed_input[col] = label_encoders[col].transform([label_encoders[col].classes_[0]])[0]  # Use most common category

    # Convert to DataFrame and add numeric values
    df_input = pd.DataFrame([processed_input])

    # Ensure correct dtype for numeric values
    for col in numeric_cols:
        df_input[col] = float(user_input[col])

    # Scale numeric features
    df_input[numeric_cols] = scaler.transform(df_input[numeric_cols])

    # Ensure the final feature order matches model training
    final_feature_order = categorical_cols + numeric_cols
    return df_input[final_feature_order].values.astype(np.float32)

In [22]:
user_input = {
    "Climate Type": "Tropical",
    "Best Visiting Season": "Winter",
    "Budget Level": "Medium",
    "Crime Index": "Low",
    "User Ratings (out of 5)": 4.5,
    "Social Media Mentions": 100000,
    "Number of Reviews": 2000
}

In [23]:
X_new = preprocess_input(user_input)